In [1]:
import smtplib
import time
import imaplib
import email
import re
from email.iterators import body_line_iterator

NAME = 'michelle.chen112453'

mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(NAME,'asdfjkl123')
mail.select('[Gmail]/Sent Mail')

type, data = mail.search(None, "ALL")
mail_ids = data[0]
id_list = mail_ids.split()
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])

from collections import defaultdict
from itertools import islice

mydict_sent = defaultdict()

for i in range(latest_email_id,first_email_id, -1):
    typ, data = mail.fetch(i, '(RFC822)' )
    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_string(data[0][1])
            #print ("SENT", msg['to'], msg['date'])
            if msg['to'] is not None : 
                m = re.match(r"staciachoe", msg['to'])
                if m is None :
                    mydict_sent[msg['to']] = msg['date']
            



In [2]:
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(NAME,'asdfjkl123')
mail.select('inbox')

type, data = mail.search(None, 'ALL')
mail_ids = data[0]
id_list = mail_ids.split()   
first_email_id = int(id_list[0])
latest_email_id = int(id_list[-1])
email_subject = list()
received_from = list()
received_date = list()
sent_date = list()
messages = list()

In [3]:
from collections import defaultdict
from itertools import islice


mydict = defaultdict(lambda: defaultdict(dict))

for i in range(latest_email_id,first_email_id, -1):
    typ, data = mail.fetch(i, '(RFC822)' )
    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_string(data[0][1])
            if msg['from'] != None :
                fullmsg = msg.get_payload(None, True)
                if msg.is_multipart() :
                    iii = 0
                    for part in msg.walk():
                        if iii == 1 :
                            if msg['from'] not in mydict :
                                messg = part.get_payload(None, True)
                                if messg is not None :
                                    messg = re.sub("\r\n", " ", messg)
                                    regex_str = r".*On (.*PM),(.*)"+re.escape(NAME)+r".*"
                                    m = re.match(regex_str, messg)
                                    if m != None :
                                        #print "MULTIPART", m.group(1), msg['from']
                                        mydict[msg['from']] = {'subject':msg['subject'], 'from':msg['from'], 'Date':msg['Date'], 'Received':m.group(1), 'message':messg}
                        iii+=1
                else :
                    #print "NOMULTIPART", msg['from']
                    tmp_from = msg['from']
                    tmp_from = re.sub(".*<", "", tmp_from) 
                    tmp_from = re.sub(">.*", "", tmp_from)
                    if tmp_from in mydict_sent :
                        #print ("GIH", msg['from'])
                        mydict[msg['from']] = {'subject':msg['subject'], 'from':msg['from'], 'Date':msg['Date'], 'Received':mydict_sent[tmp_from], 'message':fullmsg}


In [9]:
import string

f = open(NAME + '.txt','w')

def convert_date(datestring) :
    datestr = datestring
    datestr = re.sub("November", "Nov", datestr)
    datestr = re.sub("December", "Dec", datestr)
    datestr = re.sub("PM, at .*", "PM", datestr)
    datestr = re.sub("AM, at .*", "AM", datestr)
    datestr = re.sub(" at ", " ", datestr)

    
    #('BAD', 'Wed,  6 Dec 2017 14:03:04 -0800')
    if re.match(r".*, ( *)([0-9]+) (Nov|Dec) 2017 (.*) (\+|\-).*", datestr) :
        #print ("INH", datestr)
        datestr = re.sub(" (\-|\+).*", "", datestr)
        datestr = re.sub(".*, ", "", datestr)
        datestr = re.sub(" Nov ", " 11 ", datestr)
        datestr = re.sub(" Dec ", " 12 ", datestr)
        toks = datestr.split(" ")
        newdate = "%s/%s/%s %s" % (toks[1], toks[0], toks[2], toks[3])
        return newdate
    
    m = re.match(r".*(Nov|Dec) ([0-9]+), 2017(,*) (.*):(.*):(.*) (.*)", datestr)
    if m != None :
        hour = int(m.group(4))
        if m.group(7) == "PM" :
            hour += 12
        if m.group(1) == "Nov" :
            newdate = "11/" + m.group(2) + "/2017 " + str(hour) + ":" + m.group(5) + ":" + m.group(6)
        return newdate
    
    
    m = re.match(r".*(Nov|Dec) ([0-9]+), 2017(,*) (.*):(.*) (.*)", datestr)
    if m != None :
        hour = int(m.group(4))
        if m.group(6) == "PM" :
            hour += 12
        if m.group(1) == "Nov" :
            newdate = "11/" + m.group(2) + "/2017 " + str(hour) + ":" + m.group(5) + ":00"
        return newdate
    
    
    m = re.match(r"^(11/.*17)(,*) (.*):(.*) (AM|PM)", datestr)
    if m != None :
        hour = int(m.group(3))
        if m.group(5) == "PM" :
            hour += 12
        newdate = m.group(1) + " " + str(hour) + ":" + m.group(4) + ":00"
        return newdate

for i in mydict.keys() :
    received = mydict[i]['Received']
    sentdate = mydict[i]['Date']
    fromt = mydict[i]['from']
    #print (fromt, sentdate, received)
    received = re.sub(r"by.*;\r\n        ", "", received)
    received = convert_date(received)
    sentdate = convert_date(sentdate)

    messg = mydict[i]['message']
    if messg is not None :
        messg = re.sub("\r\n", " ", messg)
        messg = re.sub(r"(\-)*(\s)*Original Message.*", "", messg)
        messg = re.sub(r"On Wed, Nov 29.*", "", messg)
        messg = re.sub(r"On (Wednesday, )*November 29.*", "", messg)
        messg = re.sub(r"On (Thursday, )*November 30.*", "", messg)
        messg = re.sub(r"On Nov 29, 2017.*", "", messg)
        messg = re.sub(r"On Nov 30, 2017.*", "", messg)
        messg = re.sub(r"On 11/30/(20)*17.*", "", messg)
        messg = re.sub(r"On Thu, 11/30/(20)*17.*", "", messg)
        messg = re.sub(r"On 11/29/(20)*17.*", "", messg)
        messg = re.sub(r"On Wed, 11/29/(20)*17.*", "", messg)

        messg = re.sub(r"On Thu, Nov 30.*", "", messg)

    #print ('%s|%s|%s|%s|%s|%s' % (i, mydict[i]['subject'], mydict[i]['from'], mydict[i]['Date'], received, messg))
    f.write('%s|%s|%s|%s|%s\n' % (i, mydict[i]['subject'], received, sentdate, messg))
    
f.close()